In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from datetime import datetime

def input():

    # file input
    data = pd.read_csv('trade.csv', usecols=['uid', 'vipno', 'sldat', 'bndno'])
    data['timestamp'] = pd.to_datetime(data['sldat'])

    # sort 
    data.sort_values(['vipno','timestamp'],ascending=[1,1],inplace=True) 

    # make groups 
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0,method='first')

    # take top 60% in every group
    grouped = data.groupby(['vipno'], as_index = True).apply(lambda x: x[x['rank'] <= (0.6 * x['rank'].max())])

    # convert
    data_set = grouped.drop(['rank', 'timestamp', 'sldat', 'vipno'], axis=1).reset_index(drop=True)

    # merge by uid
    data_set['value'] = data_set['bndno']
    data_set = data_set.pivot_table(data_set, index=['uid'], columns=['bndno'])

    return data_set.fillna(0)

def input_new():

    # file input
    data = pd.read_csv('trade_new.csv', usecols=['uid', 'vipno', 'sldatime', 'bndno'])
    data['timestamp'] = pd.to_datetime(data['sldatime'])

    # sort 
    data.sort_values(['vipno','timestamp'],ascending=[1,1],inplace=True) 

    # make groups 
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0,method='first')

    # take top 60% in every group
    grouped = data.groupby(['vipno'], as_index = True).apply(lambda x: x[x['rank'] <= (0.6 * x['rank'].max())])

    # convert
    data_set = grouped.drop(['rank', 'timestamp', 'sldatime', 'vipno'], axis=1).reset_index(drop=True)

    # merge by uid
    data_set['value'] = data_set['bndno']
    data_set = data_set.pivot_table(data_set, index=['uid'], columns=['bndno'])

    return data_set.fillna(0)

def createInitSet(data_set):  

    # clear all 0
    data_array = data_set.as_matrix()
    data_dok = []
    for row in data_array:
        data_dok.append([x for x in row if x != 0.0])

    # transform array to dict
    retDict = {}  
    for trans in data_dok:  
        retDict[frozenset(trans)] = 1  

    return retDict


In [2]:
class treeNode:

    def __init__(self, name_value, num_occur, parent_node):

        self.name = name_value  
        self.count = num_occur  
        self.node_link = None  
        self.parent = parent_node  
        self.children = {}  

    def increase(self, num_occur):

        self.count += num_occur

    def disp(self, ind=1):

        print '  ' * ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind + 1)

def create_tree(data_set, min_support=1):

    """
    创建FP树
    :param data_set: 数据集
    :param min_support: 最小支持度
    :return:
    """

    freq_items = {}  # 频繁项集
    for trans in data_set:  # 第一次遍历数据集
        for item in trans:
            freq_items[item] = freq_items.get(item, 0) + data_set[trans]

    # print freq_items
    header_table = {k: v for (k, v) in freq_items.iteritems() if v >= min_support}  # 创建头指针表
    # for key in header_table:
    #     print key, header_table[key]

    # 无频繁项集
    if len(header_table) == 0:
        return None, None
    for k in header_table:
        header_table[k] = [header_table[k], None]  # 添加头指针表指向树中的数据

    # 创建树过程
    ret_tree = treeNode('Null Set', 1, None)  # 根节点

    # 第二次遍历数据集
    for trans, count in data_set.items():
        local_data = {}
        for item in trans:
            if header_table.get(item, 0):
                local_data[item] = header_table[item][0]
        if len(local_data) > 0:
            ##############################################################################################
            # 这里修改机器学习实战中的排序代码：
            ordered_items = [v[0] for v in sorted(local_data.items(), key=lambda kv: (-kv[1], kv[0]))]
            ##############################################################################################
            update_tree(ordered_items, ret_tree, header_table, count)  # populate tree with ordered freq itemset
            
    return ret_tree, header_table


def update_tree(items, in_tree, header_table, count):

    '''
    :param items: 元素项
    :param in_tree: 检查当前节点
    :param header_table:
    :param count:
    :return:
    '''
    if items[0] in in_tree.children:  # check if ordered_items[0] in ret_tree.children
        in_tree.children[items[0]].increase(count)  # incrament count
    else:  # add items[0] to in_tree.children
        in_tree.children[items[0]] = treeNode(items[0], count, in_tree)
        if header_table[items[0]][1] is None:  # update header table
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_header(header_table[items[0]][1], in_tree.children[items[0]])

    if len(items) > 1:  # call update_tree() with remaining ordered items
        update_tree(items[1::], in_tree.children[items[0]], header_table, count)


def update_header(node_test, target_node):

    '''
    :param node_test:
    :param target_node:
    :return:
    '''
    while node_test.node_link is not None:  # Do not use recursion to traverse a linked list!
        node_test = node_test.node_link

    node_test.node_link = target_node


def ascend_tree(leaf_node, pre_fix_path):

    '''
    遍历父节点，找到路径
    :param leaf_node:
    :param pre_fix_path:
    :return:
    '''
    if leaf_node.parent is not None:
        pre_fix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, pre_fix_path)


def find_pre_fix_path(base_pat, tree_node):

    '''
    创建前缀路径
    :param base_pat: 频繁项
    :param treeNode: FP树中对应的第一个节点
    :return:
    '''
    # 条件模式基
    cond_pats = {}

    while tree_node is not None:
        pre_fix_path = []
        ascend_tree(tree_node, pre_fix_path)
        if len(pre_fix_path) > 1:
            cond_pats[frozenset(pre_fix_path[1:])] = tree_node.count
        tree_node = tree_node.node_link

    return cond_pats


def mine_tree(in_tree, header_table, min_support, pre_fix, freq_items):

    '''
    挖掘频繁项集
    :param in_tree:
    :param header_table:
    :param min_support:
    :param pre_fix:
    :param freq_items:
    :return:
    '''
    # print header_table
    # 从小到大排列table中的元素，为遍历寻找频繁集合使用
    bigL = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1])]  # (sort header table)

    for base_pat in bigL:  # start from bottom of header table

        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_pat)

        # print 'finalFrequent Item: ',new_freq_set    #append to set
        if len(new_freq_set) > 0:
            freq_items[frozenset(new_freq_set)] = header_table[base_pat][0]

        cond_patt_bases = find_pre_fix_path(base_pat, header_table[base_pat][1])
        my_cond_tree, my_head = create_tree(cond_patt_bases, min_support)

        # print 'head from conditional tree: ', my_head
        if my_head is not None:  # 3. mine cond. FP-tree
            # print 'conditional tree for: ',new_freq_set
            # my_cond_tree.disp(1)
            mine_tree(my_cond_tree, my_head, min_support, new_freq_set, freq_items)


def fp_growth(data_set, min_support=1):

    my_fp_tree, my_header_tab = create_tree(data_set, min_support)
    # my_fp_tree.disp()
    freq_items = {}
    mine_tree(my_fp_tree, my_header_tab, min_support, set([]), freq_items)

    return freq_items

In [3]:
if __name__ == '__main__':

    data_set = input()

    # min_support = [2,4,8,16,32,64]
    for min_support in [2,4,8,16,32,64]:
        print "\nmin_support = %d: "%min_support
        frequent_sets = fp_growth(createInitSet(data_set), min_support).items()
        for k, v in frequent_sets:
            print "pattern:", k, "support:", v

    data_set = input_new()

    # min_support = [2,4,8,16,32,64]
    for min_support in [2,4,8,16,32,64]:
        print "\nmin_support = %d: "%min_support
        frequent_sets = fp_growth(createInitSet(data_set), min_support).items()
        for k, v in frequent_sets:
            print "pattern:", k, "support:", v



min_support = 2: 
pattern: frozenset([30248.0, 15631.0, 14069.0, 15039.0]) support: 2
pattern: frozenset([14362.0, 14123.0]) support: 2
pattern: frozenset([14442.0]) support: 11
pattern: frozenset([14224.0, 30248.0]) support: 2
pattern: frozenset([30248.0, 34079.0]) support: 2
pattern: frozenset([14076.0]) support: 2
pattern: frozenset([30248.0, 11291.0]) support: 2
pattern: frozenset([30248.0, 14622.0, 14071.0]) support: 2
pattern: frozenset([10097.0, 15083.0]) support: 3
pattern: frozenset([30248.0, 14442.0]) support: 5
pattern: frozenset([14224.0]) support: 4
pattern: frozenset([14173.0, 14622.0]) support: 2
pattern: frozenset([15045.0, 15094.0]) support: 2
pattern: frozenset([30248.0, 14258.0]) support: 3
pattern: frozenset([14051.0]) support: 14
pattern: frozenset([10094.0]) support: 8
pattern: frozenset([15092.0]) support: 30
pattern: frozenset([14341.0]) support: 4
pattern: frozenset([15012.0, 14838.0]) support: 2
pattern: frozenset([11011.0]) support: 2
pattern: frozenset([142

pattern: frozenset([14208.0, 14350.0]) support: 2
pattern: frozenset([30248.0, 10097.0, 14475.0, 15083.0]) support: 2
pattern: frozenset([10106.0, 15052.0, 10086.0]) support: 2
pattern: frozenset([14322.0, 14051.0]) support: 3
pattern: frozenset([10091.0, 15052.0, 10211.0]) support: 2
pattern: frozenset([14160.0, 30248.0]) support: 4
pattern: frozenset([30248.0, 10657.0]) support: 3
pattern: frozenset([15057.0]) support: 7
pattern: frozenset([30248.0, 14442.0, 14372.0]) support: 2
pattern: frozenset([15009.0, 14355.0, 14622.0]) support: 2
pattern: frozenset([14435.0, 15012.0]) support: 2
pattern: frozenset([30248.0, 15045.0]) support: 2
pattern: frozenset([10124.0, 15039.0]) support: 2
pattern: frozenset([15028.0, 14652.0]) support: 3
pattern: frozenset([30248.0, 10097.0]) support: 3
pattern: frozenset([15564.0]) support: 2
pattern: frozenset([14652.0]) support: 10
pattern: frozenset([10632.0, 30248.0]) support: 2
pattern: frozenset([30248.0, 10094.0]) support: 5
pattern: frozenset([10

pattern: frozenset([30248.0, 14069.0, 15631.0]) support: 2
pattern: frozenset([14627.0, 15012.0]) support: 2
pattern: frozenset([10060.0, 10094.0]) support: 2
pattern: frozenset([14539.0, 14622.0]) support: 2
pattern: frozenset([10080.0, 10007.0]) support: 2
pattern: frozenset([10124.0]) support: 7
pattern: frozenset([30248.0, 14810.0, 14366.0]) support: 2
pattern: frozenset([14265.0, 15066.0, 15038.0]) support: 2
pattern: frozenset([14442.0, 14445.0]) support: 2
pattern: frozenset([10080.0, 15012.0, 15631.0]) support: 2
pattern: frozenset([30248.0, 14338.0]) support: 6
pattern: frozenset([10106.0, 15038.0]) support: 3
pattern: frozenset([15666.0]) support: 2
pattern: frozenset([10080.0, 14044.0]) support: 2
pattern: frozenset([14705.0, 15094.0]) support: 2
pattern: frozenset([30248.0, 10097.0, 14475.0]) support: 2
pattern: frozenset([11327.0]) support: 2
pattern: frozenset([15012.0, 15094.0, 15039.0]) support: 2
pattern: frozenset([30170.0, 15631.0]) support: 3
pattern: frozenset([140

pattern: frozenset([14622.0]) support: 18
pattern: frozenset([10068.0]) support: 9
pattern: frozenset([15092.0]) support: 30
pattern: frozenset([14442.0]) support: 11
pattern: frozenset([10211.0]) support: 11
pattern: frozenset([10080.0]) support: 8
pattern: frozenset([30248.0, 15094.0]) support: 28
pattern: frozenset([30248.0, 15052.0, 15094.0]) support: 8
pattern: frozenset([15060.0]) support: 16
pattern: frozenset([14152.0]) support: 8
pattern: frozenset([15052.0]) support: 79
pattern: frozenset([30248.0, 14196.0]) support: 11
pattern: frozenset([15094.0, 15631.0]) support: 10
pattern: frozenset([14362.0]) support: 16
pattern: frozenset([30248.0, 15631.0]) support: 20
pattern: frozenset([15009.0]) support: 42
pattern: frozenset([15039.0]) support: 60
pattern: frozenset([30248.0, 15012.0]) support: 25
pattern: frozenset([30170.0]) support: 18
pattern: frozenset([15012.0, 15039.0]) support: 9
pattern: frozenset([14051.0]) support: 14
pattern: frozenset([30248.0, 14622.0]) support: 8
p

pattern: frozenset([15066.0, 14788.0]) support: 2
pattern: frozenset([14475.0, 14372.0]) support: 2
pattern: frozenset([14795.0]) support: 2
pattern: frozenset([15063.0]) support: 7
pattern: frozenset([30248.0, 14651.0, 15012.0]) support: 2
pattern: frozenset([15060.0, 15012.0]) support: 2
pattern: frozenset([15009.0, 15011.0]) support: 4
pattern: frozenset([10091.0, 10068.0]) support: 2
pattern: frozenset([14208.0, 15092.0]) support: 4
pattern: frozenset([10086.0, 15094.0, 10007.0]) support: 2
pattern: frozenset([15074.0]) support: 7
pattern: frozenset([30248.0, 14443.0]) support: 3
pattern: frozenset([10045.0, 15039.0]) support: 2
pattern: frozenset([30248.0, 14082.0, 14475.0]) support: 2
pattern: frozenset([30248.0, 10106.0, 14622.0]) support: 3
pattern: frozenset([14281.0, 14236.0]) support: 2
pattern: frozenset([15012.0, 14319.0]) support: 2
pattern: frozenset([10107.0, 10086.0]) support: 2
pattern: frozenset([11154.0, 15012.0]) support: 2
pattern: frozenset([14082.0, 14622.0]) su

...

pattern: frozenset([30248.0, 15026.0]) support: 2
pattern: frozenset([15009.0, 15052.0, 15092.0]) support: 2
pattern: frozenset([30248.0, 14322.0, 15052.0]) support: 2
pattern: frozenset([30248.0, 14393.0]) support: 3
pattern: frozenset([14032.0, 15094.0]) support: 3
pattern: frozenset([15667.0, 15012.0]) support: 2
pattern: frozenset([30248.0, 10692.0]) support: 2
pattern: frozenset([30248.0, 14475.0, 15012.0]) support: 3
pattern: frozenset([14366.0, 10086.0]) support: 3
pattern: frozenset([30248.0, 14177.0, 30170.0]) support: 2
pattern: frozenset([30248.0, 14267.0]) support: 5
pattern: frozenset([30248.0, 14723.0]) support: 2
pattern: frozenset([30248.0, 15012.0, 11079.0]) support: 3
pattern: frozenset([30248.0, 10127.0]) support: 5
pattern: frozenset([30248.0, 15009.0, 15038.0]) support: 3
pattern: frozenset([15052.0, 14838.0]) support: 3
pattern: frozenset([30248.0, 10106.0, 15045.0]) support: 4
pattern: frozenset([14622.0]) support: 56
pattern: frozenset([30248.0, 10706.0, 15012.0

pattern: frozenset([30248.0, 14177.0, 14651.0, 14678.0]) support: 2
pattern: frozenset([15094.0, 10007.0]) support: 6
pattern: frozenset([14208.0, 15094.0]) support: 3
pattern: frozenset([14648.0, 14455.0]) support: 2
pattern: frozenset([14440.0, 30248.0, 14442.0]) support: 3
pattern: frozenset([30248.0, 15009.0, 14258.0]) support: 2
pattern: frozenset([15002.0, 10106.0]) support: 2
pattern: frozenset([14444.0, 15012.0, 15094.0]) support: 2
pattern: frozenset([15012.0, 15052.0, 14044.0]) support: 2
pattern: frozenset([14208.0, 10106.0]) support: 3
pattern: frozenset([14152.0, 30248.0]) support: 2
pattern: frozenset([14098.0, 15631.0]) support: 2
pattern: frozenset([30248.0, 15094.0]) support: 80
pattern: frozenset([14442.0, 14123.0]) support: 2
pattern: frozenset([30248.0, 10730.0]) support: 2
pattern: frozenset([15632.0]) support: 7
pattern: frozenset([15123.0, 15039.0]) support: 2
pattern: frozenset([30248.0, 14362.0, 15012.0]) support: 2
pattern: frozenset([14203.0, 15052.0]) suppor

pattern: frozenset([11049.0]) support: 7
pattern: frozenset([30248.0, 10069.0]) support: 9
pattern: frozenset([10659.0]) support: 13
pattern: frozenset([30248.0, 30017.0]) support: 4
pattern: frozenset([15590.0]) support: 23
pattern: frozenset([30248.0, 15094.0, 15039.0]) support: 12
pattern: frozenset([10632.0]) support: 9
pattern: frozenset([14322.0, 15012.0]) support: 8
pattern: frozenset([14196.0, 10086.0]) support: 5
pattern: frozenset([11344.0]) support: 5
pattern: frozenset([30248.0, 14737.0]) support: 4
pattern: frozenset([30248.0, 11385.0]) support: 12
pattern: frozenset([14151.0]) support: 4
pattern: frozenset([14345.0, 15012.0]) support: 4
pattern: frozenset([30248.0, 14330.0]) support: 14
pattern: frozenset([14509.0]) support: 10
pattern: frozenset([10106.0, 10086.0]) support: 5
pattern: frozenset([14197.0]) support: 6
pattern: frozenset([14330.0]) support: 23
pattern: frozenset([30248.0, 14509.0]) support: 6
pattern: frozenset([14313.0]) support: 22
pattern: frozenset([302

pattern: frozenset([14442.0]) support: 39
pattern: frozenset([14051.0]) support: 20
pattern: frozenset([10080.0]) support: 20
pattern: frozenset([14799.0]) support: 12
pattern: frozenset([14386.0]) support: 12
pattern: frozenset([30248.0, 15012.0]) support: 71
pattern: frozenset([15001.0]) support: 13
pattern: frozenset([30248.0, 14196.0]) support: 30
pattern: frozenset([14357.0]) support: 9
pattern: frozenset([15002.0]) support: 13
pattern: frozenset([30248.0, 14051.0]) support: 9
pattern: frozenset([14224.0]) support: 12
pattern: frozenset([14102.0]) support: 9
pattern: frozenset([14333.0]) support: 12
pattern: frozenset([30248.0, 14258.0]) support: 9
pattern: frozenset([14440.0, 30248.0]) support: 8
pattern: frozenset([10211.0]) support: 23
pattern: frozenset([15092.0]) support: 64
pattern: frozenset([15052.0, 15012.0]) support: 12
pattern: frozenset([14208.0]) support: 17
pattern: frozenset([11036.0]) support: 8
pattern: frozenset([14573.0]) support: 11
pattern: frozenset([30248.0,

pattern: frozenset([14442.0]) support: 39
pattern: frozenset([14051.0]) support: 20
pattern: frozenset([10080.0]) support: 20
pattern: frozenset([30248.0, 15012.0]) support: 71
pattern: frozenset([30248.0, 14196.0]) support: 30
pattern: frozenset([15094.0, 15631.0]) support: 17
pattern: frozenset([30248.0, 14442.0]) support: 26
pattern: frozenset([15123.0]) support: 21
pattern: frozenset([15092.0]) support: 64
pattern: frozenset([14208.0]) support: 17
pattern: frozenset([30248.0, 14366.0]) support: 16
pattern: frozenset([10060.0]) support: 43
pattern: frozenset([15052.0, 15094.0]) support: 20
pattern: frozenset([14366.0]) support: 44
pattern: frozenset([30248.0, 30170.0]) support: 19
pattern: frozenset([15590.0]) support: 23
pattern: frozenset([14079.0]) support: 16
pattern: frozenset([15038.0]) support: 33
pattern: frozenset([15045.0]) support: 33
pattern: frozenset([30248.0, 14177.0]) support: 26
pattern: frozenset([14098.0]) support: 49
pattern: frozenset([30248.0]) support: 507
pat